In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import col, expr, round
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType, IntegerType
from pyspark.sql.functions import format_number

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

23/11/08 14:39:03 WARN Utils: Your hostname, Ellas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.80.80.78 instead (on interface en0)
23/11/08 14:39:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/08 14:39:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:

# Convert the "date" column to a date type
df = df.withColumn("date", to_date(df["date"]))
df = df.withColumn("date_built", to_date(df["date_built"]))

# Extract the year from the "date" and "date_built" columns
df_with_year = df.withColumn("Year_of_sale", year(df["date"])).withColumn("Year_built", year(df["date_built"]))

# Show the DataFrame with the extracted year
df_with_year.show()




+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+------------+----------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|Year_of_sale|Year_built|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+------------+----------+
|f8a53099-ba1c-47d...|2022-04-08|2016-01-01|936923|       4|        3|       3167|   11733|     2|         1|  76|        2022|      2016|
|7530a2d8-1ae3-451...|2021-06-13|2013-01-01|379628|       2|        2|       2235|   14384|     1|         0|  23|        2021|      2013|
|43de979c-0bf0-4c9...|2019-04-12|2014-01-01|417866|       2|        2|       2127|   10575|     2|         0|   0|        2019|      2014|
|b672c137-b88c-48b...|2019-10-16|2016-01-01|239895|       2|        2|       1631|   11149|     2|         0|   0|        2019|      2016|
|e0726d4d-d595-407...|2022-

In [7]:
## modify table schema

df_with_year_with_modified_schema = df_with_year.withColumn("price", col("price").cast(FloatType())) \
    .withColumn("sqft_living", col("sqft_living").cast(IntegerType())) \
    .withColumn("sqft_lot", col("sqft_lot").cast(IntegerType())) \
.withColumn("bedrooms", col("bedrooms").cast(IntegerType())) \
.withColumn("bathrooms", col("bathrooms").cast(IntegerType())) \
    .withColumn("floors", col("floors").cast(IntegerType()))


df_with_year_with_modified_schema.printSchema()

df2 =df_with_year_with_modified_schema 

# Format the "price" column with 2 decimal places
#df2 = df2.withColumn("price", format_number(df2["price"], 2))
df2.show()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: date (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- Year_of_sale: integer (nullable = true)
 |-- Year_built: integer (nullable = true)

+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+------------+----------+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|Year_of_sale|Year_built|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+------------+----------+
|f8a53099-ba1c-47d...|2022-04-08|2016-01-01|936923.0| 

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# Register the DataFrame as a temporary view
df2.createOrReplaceTempView("house_sales")



query = """
SELECT Year_of_sale, ROUND(AVG(price), 2) as avg_price
FROM house_sales
WHERE bedrooms = 4
GROUP BY Year_of_sale
"""
spark.sql(query).show()



+------------+---------+
|Year_of_sale|avg_price|
+------------+---------+
|        2022|296363.88|
|        2019| 300263.7|
|        2020|298353.78|
|        2021|301819.44|
+------------+---------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Register the DataFrame as a temporary view
df2.createOrReplaceTempView("house_sales")

query = """
SELECT date_built, ROUND(AVG(price), 2) as avg_price
FROM house_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

spark.sql(query).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|2010-01-01|292859.62|
|2011-01-01|291117.47|
|2012-01-01|293683.19|
|2013-01-01|295962.27|
|2014-01-01|290852.27|
|2015-01-01| 288770.3|
|2016-01-01|290555.07|
|2017-01-01|292676.79|
+----------+---------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Register the DataFrame as a temporary view
df2.createOrReplaceTempView("house_sales")

query = """
SELECT date_built, ROUND(AVG(price), 2) as avg_price
FROM house_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 
GROUP BY date_built
ORDER BY date_built
"""

spark.sql(query).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|2010-01-01|285010.22|
|2011-01-01|276553.81|
|2012-01-01|307539.97|
|2013-01-01|303676.79|
|2014-01-01|298264.72|
|2015-01-01|297609.97|
|2016-01-01| 293965.1|
|2017-01-01|280317.58|
+----------+---------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Register the DataFrame as a temporary view

df2.createOrReplaceTempView("house_sales")

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) as avg_price
FROM house_sales
WHERE price >= 350000
GROUP BY view
ORDER BY VIEW 

"""

spark.sql(query).show()





print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.6378309726715088 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
home_sales = df2
home_sales.createOrReplaceTempView("home_sales")

spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY VIEW 

"""

spark.sql(query).show()






print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.5363688468933105 seconds ---


The cached query on the cached file(home_sales) ran faster than the query on the uncashed file  ( .63 seconds vs .57 seconds)

In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

# Partition our data by UniqueCarrier
home_sales.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_delayed")



In [16]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('home_sales_delayed')
p_df.show()

+--------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+------------+----------+----------+
|                  id|      date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|Year_of_sale|Year_built|date_built|
+--------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+------------+----------+----------+
|2ed8d509-7372-46d...|2021-08-06|258710.0|       3|        3|       1918|    9666|     1|         0|  25|        2021|      2015|2015-01-01|
|941bad30-eb49-4a7...|2020-05-09|229896.0|       3|        3|       2197|    8641|     1|         0|   3|        2020|      2015|2015-01-01|
|c797ca12-52cd-4b1...|2019-06-08|288650.0|       2|        3|       2100|   10419|     2|         0|   7|        2019|      2015|2015-01-01|
|0cfe57f3-28c2-472...|2019-10-04|308313.0|       3|        3|       1960|    9453|     2|         0|   2|        2019|      2015|2015-01-01|
|d715f295-2fb

In [17]:
# 12. Create a temporary table for the parquet data.

p_df.createOrReplaceTempView("home_sales_delayed_table")


In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 
start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) as avg_price
FROM home_sales_delayed_table
WHERE price >= 350000
GROUP BY view
ORDER BY VIEW 

"""

spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 1.7523009777069092 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [23]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False